We see that there are many missing values and they can be taken from the internet. I can proceed in that direction.

In [22]:
import pandas as pd
import numpy as np
import mysql.connector

In [23]:
df_netflix_movie_data = pd.read_csv('./Datasets/netflix_titles.csv', lineterminator = '\n')
df_prime_movie_data = pd.read_csv('./Datasets/amazon_prime_titles.csv', lineterminator = '\n')
df_disney_plus_movie_data = pd.read_csv('./Datasets/disney_plus_titles.csv', lineterminator = '\n')
df_hulu_movie_data = pd.read_csv('./Datasets/hulu_titles.csv', lineterminator = '\n')

In [ ]:
display(df_netflix_movie_data)

In [ ]:
missing_value_netflix = df_netflix_movie_data.isna().sum()
print(missing_value_netflix)

In [ ]:
print("Number of records in Netflix DataFrame:", df_netflix_movie_data.shape[0])

In [ ]:
missing_value_prime = df_prime_movie_data.isna().sum()
print(missing_value_prime)

In [ ]:
print("Number of records in Prime Video DataFrame:", df_prime_movie_data.shape[0])

In [ ]:
missing_value_disney_plus = df_disney_plus_movie_data.isna().sum()
print(missing_value_disney_plus)

In [ ]:
print("Number of records in Disney Plus DataFrame:", df_disney_plus_movie_data.shape[0])

In [ ]:
missing_value_hulu = df_hulu_movie_data.isna().sum()
print(missing_value_hulu)

In [ ]:
print("Number of records in Hulu DataFrame:", df_hulu_movie_data.shape[0])

As we can see clearly, above only columns director, cast, country, date_added, rating, duration. Rather than deleting them. If we can get those values back in the dataset it would be best.
I can use information present on the website imdb to get the data and fill it. I can use web scrapping to get the data and fill it in the dataset.

But before that I can check data. One dataset row may have data of the for the other dataset missing row values. Let's first check for the 

In [ ]:
print("Netflix DataFrame Column Data Types:")
print(df_netflix_movie_data.dtypes)

In [ ]:
print("\nPrime Video DataFrame Column Data Types:")
print(df_prime_movie_data.dtypes)

In [ ]:

print("\nDisney Plus DataFrame Column Data Types:")
print(df_disney_plus_movie_data.dtypes)

In [ ]:
print("\nHulu DataFrame Column Data Types:")
print(df_hulu_movie_data.dtypes)

Okay I will be first converting the datatypes of the columns of the dataframes according to the requirement and then after that I will be tranferring data to the SQL. Through this I can ensure that there is no error while tranferring the data.

Why I want to transfer the data to SQL?
I currently want to fill the missing values of the columns director, cast, country, date_added, rating and duration in all datasets. But there are chances that missing value of 1 dataset might be present in another dataset. So for doing that in optimized manner I am using the SQL. 
Other approaches:

- I can loop through datasets to find values. But this approach will take most time as there are 4 datasets and time taken will get to O($n^4$). This approach is not great.
- I can use Hash tables for this but it will be take most of the memory for it. It would be better to go for Database as I would be taking it's help for data wrangling also.

In [24]:
dataframes = [df_netflix_movie_data, df_prime_movie_data, df_disney_plus_movie_data, df_hulu_movie_data]

In [25]:
# Combine all datasets
df_combined = pd.concat(dataframes, ignore_index=True)


In [ ]:
display(df_combined)

In [26]:
# 1. Select one row where any value is null
row_with_any_null = df_combined[df_combined.isnull().any(axis=1)].head(1)

In [ ]:
display(row_with_any_null)

In [27]:
# 2. Select one row where specific columns are null
row_with_all_specific_null = df_combined[
    df_combined[['director', 'cast', 'country', 'date_added', 'rating', 'duration', 'description']].isnull().all(axis=1)
].head(1)


In [9]:
display(row_with_all_specific_null)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description


In [28]:
# 3. Select rows where only one column at a time is null
rows_with_one_null = []
columns_to_check = ['director', 'cast', 'country', 'date_added', 'rating', 'duration', 'description']

for col in columns_to_check:
    row = df_combined[df_combined[col].isnull() & df_combined.drop(columns=[col]).notnull().all(axis=1)].head(1)
    rows_with_one_null.append(row)

# Combine all selected rows into a new DataFrame
df_missing_data_check = pd.concat([row_with_any_null, row_with_all_specific_null] + rows_with_one_null, ignore_index=True)

In [ ]:
# Display the resulting DataFrame
display(df_missing_data_check)

In [ ]:
df_missing_data_check.describe

In [29]:
# Ensure country column contains only one country per movie
for df in dataframes:
    if 'country' in df.columns:
        df['country'] = df['country'].astype(str).apply(lambda x: x.split(',')[0].strip() if pd.notnull(x) else x)

print("[INFO] Updated country column to keep only the first country for each movie.")

[INFO] Updated country column to keep only the first country for each movie.


In [30]:
for df in dataframes:
    df["show_id"] = df["show_id"].astype("string")  # Explicit string type
    df["type"] = df["type"].astype("string")
    df["title"] = df["title"].astype("string")
    df["director"] = df["director"].astype("string")
    df["cast"] = df["cast"].astype("string")  # Convert float64 to string
    df["country"] = df["country"].astype("string")
    df["date_added"] = pd.to_datetime(df["date_added"], errors="coerce")  # Convert to Date
    df["release_year"] = df["release_year"].astype("Int64")  # Explicitly use Int64
    df["rating"] = df["rating"].astype("string")
    df["duration"] = df["duration"].astype("string")
    df["listed_in"] = df["listed_in"].astype("string")
    df["description"] = df["description"].astype("string")



In [31]:
# Verify Data Types
total_num_of_rows = 0
for i, df in enumerate(dataframes, start=1):
    total_num_of_rows += len(df)
    print(f"\nData Types for DataFrame {i}:\n", df.dtypes)

print(f"\n Total number of rows:{total_num_of_rows}")   


Data Types for DataFrame 1:
 show_id         string[python]
type            string[python]
title           string[python]
director        string[python]
cast            string[python]
country         string[python]
date_added      datetime64[ns]
release_year             Int64
rating          string[python]
duration        string[python]
listed_in       string[python]
description     string[python]
dtype: object

Data Types for DataFrame 2:
 show_id         string[python]
type            string[python]
title           string[python]
director        string[python]
cast            string[python]
country         string[python]
date_added      datetime64[ns]
release_year             Int64
rating          string[python]
duration        string[python]
listed_in       string[python]
description     string[python]
dtype: object

Data Types for DataFrame 3:
 show_id         string[python]
type            string[python]
title           string[python]
director        string[python]
cast          

In [38]:
len(dataframes)

4

In [ ]:
display(dataframes[0])

In [32]:
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="Himanshu",
    password="12345",
)
cursor = conn.cursor()

In [33]:
cursor.execute("USE movies_db")  # Switch to a different DB
cursor.execute("DROP DATABASE IF EXISTS movies_db")
conn.commit()

In [34]:
cursor.execute("CREATE DATABASE IF NOT EXISTS movies_db;")
cursor.close()
conn.close()

In [35]:
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="Himanshu",
    password="12345",
    database="movies_db"
)
cursor = conn.cursor()

In [36]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS movies_data (
        show_id VARCHAR(255),
        type VARCHAR(50),
        title VARCHAR(500),
        director VARCHAR(300),
        cast VARCHAR(2000),
        country VARCHAR(100),
        date_added DATE,
        release_year YEAR,
        rating VARCHAR(50),
        duration VARCHAR(100),
        listed_in VARCHAR(300),
        description TEXT,
        PRIMARY KEY (title, release_year)
    )
""")

Okay so the database has been created. But I have to make sure that all the data goes properly inside the database.

In [ ]:
import os



In [ ]:
# List to store failed insertions
failed_insertions = []

# Define table name
table_name = "movies_data"

# Iterate through the DataFrame for insertion
for index, row in df.iterrows():
    try:
        # Convert NaN to None for SQL compatibility
        values = [None if pd.isna(row[col]) else row[col] for col in df.columns]
        
        # Construct INSERT query dynamically
        columns_str = ", ".join(df.columns)
        placeholders = ", ".join(["%s"] * len(df.columns))
        insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})"

        # Log data before insertion
        print(f"[INFO] Inserting data: {values}")

        # Execute insertion
        cursor.execute(insert_query, tuple(values))
        conn.commit()
        print("[SUCCESS] Data inserted successfully!")

        # Fetch inserted data to validate
        primary_keys = ["title", "release_year"]
        where_clause = " AND ".join([f"{pk} = %s" for pk in primary_keys])
        select_query = f"SELECT * FROM {table_name} WHERE {where_clause}"

        cursor.execute(select_query, tuple(row[pk] for pk in primary_keys))
        db_record = cursor.fetchone()

        # If no matching record found, log missing data
        if not db_record:
            print(f"[ERROR] Data missing from DB after insertion: {row[primary_keys].to_dict()}")
            failed_insertions.append({**row.to_dict(), "error": "Data missing after insertion"})
            continue

        # Compare column values
        db_dict = dict(zip(df.columns, db_record))
        for col in df.columns:
            df_value = str(row[col]).strip() if pd.notna(row[col]) else None
            db_value = str(db_dict[col]).strip() if db_dict[col] is not None else None

            if df_value != db_value:
                print(f"[WARNING] Mismatch found in column '{col}': DF='{df_value}', DB='{db_value}'")
                failed_insertions.append({**row.to_dict(), "error": f"Mismatch in column '{col}'"})

    except Exception as e:
        print(f"[ERROR] Failed to insert data: {row.to_dict()}\nReason: {str(e)}")
        failed_insertions.append({**row.to_dict(), "error": str(e)})
        conn.rollback()  # Rollback on failure

# Convert failed insertions into DataFrame
df_failed_insertions = pd.DataFrame(failed_insertions)

In [ ]:
# Initialize failed_inserts as an empty DataFrame with correct columns
df_failed_inserts = pd.DataFrame(columns=df.columns)

# Assuming you have already established a connection and cursor
cursor = conn.cursor()

# Insert each row, perform verification, and handle errors
for _, row in df.iterrows():
    try:
        # Convert NaN values to None, which can be handled by MySQL
        row = row.where(pd.notna(row), None)

        # Log the row that is about to be inserted
        print(f"\nInserting row with title: {row['title']} and release_year: {row['release_year']}")

        # Insert data into the database
        insert_query = """
            INSERT INTO movies_data 
            (show_id, type, title, director, cast, country, date_added, release_year, 
             rating, duration, listed_in, description)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (
            row["show_id"], row["type"], row["title"], row["director"],
            row["cast"], row["country"], row["date_added"], row["release_year"],
            row["rating"], row["duration"], row["listed_in"], row["description"]
        ))
        conn.commit()  # Commit the transaction

        # Log success
        print(f"Data for '{row['title']}' inserted successfully.")

    except mysql.connector.Error as err:
        # Log the error
        print(f"\nError occurred while inserting data: {err}")
        print(f"Row causing the error: \n{row.to_frame().T}")  # Print row in tabular format

        # Append failed row properly to df_failed_inserts
        df_failed_inserts = pd.concat([df_failed_inserts, row.to_frame().T], ignore_index=True)

# Print failed insertions, if any
if not df_failed_inserts.empty:
    print("\nFailed Insertions due to duplicates or errors: ")
    print(df_failed_inserts)
else:
    print("\nAll rows inserted successfully without errors.")

In [ ]:
display(df_failed_inserts)

In [25]:
# 1. Compute Total Rows Before Insertion
total_num_of_rows_in_df = (
    len(df_netflix_movie_data) + 
    len(df_prime_movie_data) + 
    len(df_disney_plus_movie_data) + 
    len(df_hulu_movie_data)
)

# 2. Remove Duplicate Rows That Failed to Insert
total_num_of_rows_expected_in_db = total_num_of_rows_in_df - len(df_failed_inserts)

cursor.execute("SELECT COUNT(*) FROM movies_data;")
total_num_of_rows_in_db = cursor.fetchone()[0]

# 4. Validate Data Transfer
if total_num_of_rows_expected_in_db == total_num_of_rows_in_db:
    print("[SUCCESS] No data loss!")
else:
    print("[WARNING] Data loss detected!")
    print(f"Expected in DB: {total_num_of_rows_expected_in_db}, Found in DB: {total_num_of_rows_in_db}")

[WARNING] Data loss detected!
Expected in DB: 22996, Found in DB: 3071


In [31]:
cursor = conn.cursor()

# Function to update the database with missing values filled
def update_missing_values(row):
    try:
        # Extract the row data from the database using Title and Release Year
        cursor.execute("""
            SELECT * FROM movies_data WHERE title = %s AND release_year = %s
        """, (row["title"], row["release_year"]))
        
        db_row = cursor.fetchone()

        if db_row:
            # Convert fetched row into a dictionary
            columns = ["show_id", "type", "title", "director", "cast", "country", 
                       "date_added", "release_year", "rating", "duration", 
                       "listed_in", "description"]
            db_data = dict(zip(columns, db_row))

            # Compare each element and fill missing values
            updated_data = {}
            for col in columns:
                if pd.isna(row[col]) or row[col] is None:
                    updated_data[col] = db_data[col]  # Fill missing value from DB
                else:
                    updated_data[col] = row[col]  # Keep the original value if not missing
            
            # If any values were updated, modify the database
            update_query = """
                UPDATE movies_data
                SET show_id = %s, type = %s, director = %s, cast = %s, country = %s, 
                    date_added = %s, rating = %s, duration = %s, listed_in = %s, 
                    description = %s
                WHERE title = %s AND release_year = %s
            """
            cursor.execute(update_query, (
                updated_data["show_id"], updated_data["type"], updated_data["director"], 
                updated_data["cast"], updated_data["country"], updated_data["date_added"], 
                updated_data["rating"], updated_data["duration"], updated_data["listed_in"], 
                updated_data["description"], updated_data["title"], updated_data["release_year"]
            ))
            conn.commit()
            print(f"Updated missing values for: {row['title']} ({row['release_year']})")
        else:
            print(f"No matching record found in database for: {row['title']} ({row['release_year']})")

    except mysql.connector.Error as err:
        print(f"Error updating {row['title']} ({row['release_year']}): {err}")

# Loop through each row in the dataframe and process
for _, row in df_failed_inserts.iterrows():
    update_missing_values(row)

print("Data processing complete.")

Updated missing values for: Trolls: TrollsTopia (2020)
No matching record found in database for: Monos (2019)
Data processing complete.


In [ ]:
# List to store records with mismatched data
mismatch_data = []

# Get the list of columns in df_missing_data_check (excluding primary keys)
columns_to_check = list(df_missing_data_check.columns)
columns_to_check.remove("title")
columns_to_check.remove("release_year")

# Iterate through df_missing_data_check to verify each record
for index, row in df_missing_data_check.iterrows():
    title = row["title"]
    release_year = row["release_year"]

    # Query to fetch the corresponding row from the database
    query = f"""
    SELECT {', '.join(columns_to_check)} FROM your_table_name 
    WHERE title = %s AND release_year = %s;
    """
    
    cursor.execute(query, (title, release_year))
    result = cursor.fetchone()

    # If no matching record found, log as missing
    if not result:
        mismatch_data.append({
            "title": title,
            "release_year": release_year,
            "column": "ALL",
            "df_value": "Exists in DF",
            "db_value": "Missing in DB"
        })
        continue

    # Compare each column value
    db_row = dict(zip(columns_to_check, result))
    for col in columns_to_check:
        df_value = str(row[col]).strip() if pd.notna(row[col]) else None
        db_value = str(db_row[col]).strip() if db_row[col] is not None else None

        if df_value != db_value:
            mismatch_data.append({
                "title": title,
                "release_year": release_year,
                "column": col,
                "df_value": row[col],
                "db_value": db_row[col]
            })

# Convert mismatched records to a DataFrame
df_mismatched_data = pd.DataFrame(mismatch_data, columns=["title", "release_year", "column", "df_value", "db_value"])

# Display results
if df_mismatched_data.empty:
    print("[SUCCESS] All records in df_missing_data_check match exactly in the database. 🎉")
else:
    print("[WARNING] Some records in df_missing_data_check do not match the database!")
    print(df_mismatched_data)